In [2]:
import math
import cv2
import numpy as np, cv2

width = 640
height = 480

referencePoints = [np.float32([[width/8,height/8],[1*width/4,height/8],[1*width/4,1*height/4],[width/8,1*height/4]]), 
np.float32([[width/4,height/2],[3*width/8,height/2],[3*width/8,3*height/4],[width/4,3*height/4]]), 
np.float32([[width/2,2*height/5],[3*width/4,height/2],[3*width/4,3*height/4],[width/2,2*height/4]])]

currentPoint = [-1,-1,-1]

calibrating = True
fullScreen = False

cap, cap2, cap3 = cv2.VideoCapture('ret.mp4'), cv2.VideoCapture('orochi.avi'), cv2.VideoCapture('notorius.mp4')
(ret, frame), (ret2, frame2), (ret3, frame3) = cap.read(), cap2.read(), cap3.read()

#inputimage1 = cv2.imread("pp.jpg")
#inputimage2 = cv2.imread("wallis.jpeg")
#inputimage3 = cv2.imread("ronaldo.jpg")

inputimage1, inputimage2, inputimage3 = frame, frame2, frame3

rows1, cols1 = inputimage1.shape[:2]
pts1 = np.float32([[0,0],[cols1,0],[cols1,rows1],[0,rows1]])

rows2, cols2 = inputimage2.shape[:2]
pts2 = np.float32([[0,0],[cols2,0],[cols2,rows2],[0,rows2]])

rows3, cols3 = inputimage3.shape[:2]
pts3 = np.float32([[0,0],[cols3,0],[cols3,rows3],[0,rows3]])

image = np.zeros((height, width, 3), np.uint8)

def pointColor(n):
    if n == 0:
        return (0,0,255)
    elif n == 1:
        return (0,255,255)
    elif n == 2:
        return (255,255,0)
    else:
        return (0,255,0)

def mouse(event, x, y, flags, param):
    global currentPoint 

    if event == cv2.EVENT_LBUTTONDOWN:
        cp, cp2, cp3 = 0,0,0
        
        for point in referencePoints[0]:
            dist = math.sqrt((x-point[0])*(x-point[0])+(y-point[1])*(y-point[1]))
            if dist < 4:
                currentPoint[0] = cp
                break
            else:
                cp = cp + 1

        for point in referencePoints[1]:
            dist = math.sqrt((x-point[0])*(x-point[0])+(y-point[1])*(y-point[1]))
            if dist < 4:
                currentPoint[1] = cp2
                break
            else:
                cp2 = cp2 + 1

        for point in referencePoints[2]:
            dist = math.sqrt((x-point[0])*(x-point[0])+(y-point[1])*(y-point[1]))
            if dist < 4:
                currentPoint[2] = cp3
                break
            else:
                cp3 = cp3 + 1
        
    if event == cv2.EVENT_LBUTTONUP:
        currentPoint = [-1,-1,-1]

    if currentPoint[0] != -1:
        referencePoints[0][currentPoint[0]] = [x,y]

    if currentPoint[1] != -1:
        referencePoints[1][currentPoint[1]] = [x,y]

    if currentPoint[2] != -1:
        referencePoints[2][currentPoint[2]] = [x,y]
    
cv2.namedWindow("test", cv2.WINDOW_NORMAL)
cv2.setMouseCallback("test", mouse)

while True:
    
    image[:] = (0,0,0)
    
    (ret, frame), (ret2, frame2), (ret3, frame3) = cap.read(), cap2.read(), cap3.read()

    inputimage1, inputimage2, inputimage3 = frame, frame2, frame3
    
    if calibrating:
        color = 0
        for point in referencePoints[0]:
            cv2.circle(image, (int(point[0]), int(point[1])),5,pointColor(color), -1)
            color = color + 1
        color2 = 0
        for point in referencePoints[1]:
            cv2.circle(image, (int(point[0]), int(point[1])),5,pointColor(color2), -1)
            color2 = color2 + 1
        color3 = 0
        for point in referencePoints[2]:
            cv2.circle(image, (int(point[0]), int(point[1])),5,pointColor(color3), -1)
            color3 = color3 + 1

            
    M = cv2.getPerspectiveTransform(pts1,referencePoints[0])
    M2 = cv2.getPerspectiveTransform(pts2,referencePoints[1])
    M3 = cv2.getPerspectiveTransform(pts3,referencePoints[2])

    
    cv2.warpPerspective(inputimage1, M, (width,height), image, borderMode=cv2.BORDER_TRANSPARENT)
    cv2.warpPerspective(inputimage2, M2, (width,height), image, borderMode=cv2.BORDER_TRANSPARENT)
    cv2.warpPerspective(inputimage3, M3, (width,height), image, borderMode=cv2.BORDER_TRANSPARENT)

    
    cv2.imshow("test", image)
    key = cv2.waitKey(25) & 0xFF

    if key == 27:
        break
        
    if key == ord("c"):
        calibrating = not calibrating

    if key == ord("f"):
        if fullScreen == False:
            cv2.setWindowProperty("test", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        else:
            cv2.setWindowProperty("test", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
        fullScreen = not fullScreen

cv2.destroyAllWindows()